#### 1. example

In [2]:
import cirq
import numpy as np

In [4]:
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1)
)
circuit

0: ───H───@───M───
          │   │
1: ───────X───M───

In [10]:
print(circuit.to_text_diagram(use_unicode_characters=True))

0: ───H───@───M───
          │   │
1: ───────X───M───


In [11]:
import sys
print(sys.path)

['/home/hongxiangzhu/.pyenv/versions/3.11.8/lib/python311.zip', '/home/hongxiangzhu/.pyenv/versions/3.11.8/lib/python3.11', '/home/hongxiangzhu/.pyenv/versions/3.11.8/lib/python3.11/lib-dynload', '', '/home/hongxiangzhu/.pyenv/versions/cirq-dev/lib/python3.11/site-packages']


In [1]:
!pip list | grep cirq

cirq-aqt                  1.6.0.dev0              /home/hongxiangzhu/cirq_project/dev/Cirq/cirq-aqt
cirq-core                 1.6.0.dev0              /home/hongxiangzhu/cirq_project/dev/Cirq/cirq-core
cirq-google               1.6.0.dev0              /home/hongxiangzhu/cirq_project/dev/Cirq/cirq-google
cirq-ionq                 1.6.0.dev0              /home/hongxiangzhu/cirq_project/dev/Cirq/cirq-ionq
stimcirq                  1.15.0


In [2]:
import sys
for p in sys.path:
    print(p)


/home/hongxiangzhu/.pyenv/versions/3.11.8/lib/python311.zip
/home/hongxiangzhu/.pyenv/versions/3.11.8/lib/python3.11
/home/hongxiangzhu/.pyenv/versions/3.11.8/lib/python3.11/lib-dynload

/home/hongxiangzhu/.pyenv/versions/cirq-dev/lib/python3.11/site-packages
/home/hongxiangzhu/cirq_project/dev/Cirq/cirq-core
/home/hongxiangzhu/cirq_project/dev/Cirq/cirq-ionq
/home/hongxiangzhu/cirq_project/dev/Cirq/cirq-aqt
/home/hongxiangzhu/cirq_project/dev/Cirq/cirq-google


In [4]:
import cirq_google

print(cirq_google.Sycamore)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

#### 2. basic test
##### 2.1 Circuits

In [7]:
qubits = cirq.GridQubit.square(3)

print(qubits[0])
print(qubits)

q(0, 0)
[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [8]:
# This is a Pauli X gate. It is an object instance.
x_gate = cirq.X
# Applying it to the qubit at location (0, 0) (defined above)
# turns it into an operation.
x_op = x_gate(qubits[0])

print(x_op)

X(q(0, 0))


In [9]:
cz = cirq.CZ(qubits[0], qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment(x, cz)

print(moment)

  ╷ 0 1 2
╶─┼───────
0 │ @─@ X
  │


In [10]:
cz01 = cirq.CZ(qubits[0], qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1], qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])
circuit = cirq.Circuit((moment0, moment1))

print(circuit)

(0, 0): ───@───────
           │
(0, 1): ───@───@───
               │
(0, 2): ───X───@───


In [11]:
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]
circuit = cirq.Circuit()
circuit.append([cirq.CZ(q0, q1), cirq.H(q2)])

print(circuit)

(0, 0): ───@───
           │
(1, 0): ───@───

(2, 0): ───H───


In [12]:
circuit.append([cirq.H(q0), cirq.CZ(q1, q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


In [13]:
circuit = cirq.Circuit()
circuit.append([cirq.CZ(q0, q1), cirq.H(q2), cirq.H(q0), cirq.CZ(q1, q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


This has again created two Moment objects. How did Circuit know how to do this? The Circuit.append method (and its cousin, Circuit.insert) both take an argument called strategy of type cirq.InsertStrategy. By default, InsertStrategy is InsertStrategy.EARLIEST.

In [14]:
from cirq.circuits import InsertStrategy

circuit = cirq.Circuit()
circuit.append([cirq.CZ(q0, q1)])
circuit.append([cirq.H(q0), cirq.H(q2)], strategy=InsertStrategy.EARLIEST)

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───H───────


In [15]:
circuit = cirq.Circuit()
circuit.append([cirq.H(q0), cirq.H(q1), cirq.H(q2)], strategy=InsertStrategy.NEW)

print(circuit)

(0, 0): ───H───────────

(1, 0): ───────H───────

(2, 0): ───────────H───


In [16]:
circuit = cirq.Circuit()
circuit.append([cirq.CZ(q1, q2)])
circuit.append([cirq.CZ(q1, q2)])
circuit.append([cirq.H(q0), cirq.H(q1), cirq.H(q2)], strategy=InsertStrategy.INLINE)

print(circuit)

(0, 0): ───────────H───

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───


In [17]:
circuit = cirq.Circuit()
circuit.append([cirq.H(q0)])
circuit.append([cirq.CZ(q1, q2), cirq.H(q0)], strategy=InsertStrategy.NEW_THEN_INLINE)

print(circuit)

(0, 0): ───H───H───

(1, 0): ───────@───
               │
(2, 0): ───────@───


In [23]:
def my_layer():
    yield [cirq.CZ(q0, q1)]
    yield [cirq.H(q) for q in (q0, q1, q2)]
    yield [cirq.CZ(q1, q2)]
    yield [cirq.H(q0), [cirq.CZ(q1, q2)]]


circuit = cirq.Circuit()
circuit.append(my_layer())

for x in my_layer():
    print(x)

print(circuit)

[cirq.CZ(cirq.GridQubit(0, 0), cirq.GridQubit(1, 0))]
[cirq.H(cirq.GridQubit(0, 0)), cirq.H(cirq.GridQubit(1, 0)), cirq.H(cirq.GridQubit(2, 0))]
[cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H(cirq.GridQubit(0, 0)), [cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]
(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


In [24]:
circuit = cirq.Circuit(cirq.H(q0), cirq.H(q1))
print(circuit)

(0, 0): ───H───

(1, 0): ───H───


In [25]:
circuit = cirq.Circuit(cirq.H(q0), cirq.CZ(q0, q1))
for moment in circuit:
    print(moment)

  ╷ 0
╶─┼───
0 │ H
  │
  ╷ 0
╶─┼───
0 │ @
  │ │
1 │ @
  │


In [38]:
circuit = cirq.Circuit(cirq.H(q0), cirq.CZ(q0, q1), cirq.H(q1), cirq.CZ(q0, q1))
print(circuit[1:3])
print(circuit[0:2])
print(circuit[:-1])
print(circuit[::2])

(0, 0): ───@───────
           │
(1, 0): ───@───H───
(0, 0): ───H───@───
               │
(1, 0): ───────@───
(0, 0): ───H───@───────
               │
(1, 0): ───────@───H───
(0, 0): ───H───────

(1, 0): ───────H───


In [39]:
subcircuit = cirq.Circuit(cirq.H(q1), cirq.CZ(q0, q1), cirq.CZ(q2, q1), cirq.H(q1))
subcircuit_op = cirq.CircuitOperation(subcircuit.freeze())
circuit = cirq.Circuit(cirq.H(q0), cirq.H(q2), subcircuit_op)
print(circuit)

               [ (0, 0): ───────@─────────── ]
               [                │            ]
(0, 0): ───H───[ (1, 0): ───H───@───@───H─── ]───
               [                    │        ]
               [ (2, 0): ───────────@─────── ]
               │
(1, 0): ───────#2────────────────────────────────
               │
(2, 0): ───H───#3────────────────────────────────


In [40]:
circuit = cirq.Circuit(
    cirq.CircuitOperation(
        cirq.FrozenCircuit(cirq.H(q1), cirq.CZ(q0, q1), cirq.CZ(q2, q1), cirq.H(q1))
    )
)
print(circuit)

           [ (0, 0): ───────@─────────── ]
           [                │            ]
(0, 0): ───[ (1, 0): ───H───@───@───H─── ]───
           [                    │        ]
           [ (2, 0): ───────────@─────── ]
           │
(1, 0): ───#2────────────────────────────────
           │
(2, 0): ───#3────────────────────────────────


In [41]:
subcircuit_op = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.CZ(q0, q1)))

# Create a copy of subcircuit_op that repeats twice...
repeated_subcircuit_op = subcircuit_op.repeat(2)

# ...and another copy that replaces q0 with q2 to perform CZ(q2, q1).
moved_subcircuit_op = subcircuit_op.with_qubit_mapping({q0: q2})
circuit = cirq.Circuit(repeated_subcircuit_op, moved_subcircuit_op)
print(circuit)

           [ (0, 0): ───@─── ]
(0, 0): ───[            │    ]────────────────────────────────────────────────────────────────
           [ (1, 0): ───@─── ](loops=2)
           │
(1, 0): ───#2─────────────────────────────#2──────────────────────────────────────────────────
                                          │
                                          [ (0, 0): ───@─── ]
(2, 0): ──────────────────────────────────[            │    ]─────────────────────────────────
                                          [ (1, 0): ───@─── ](qubit_map={q(0, 0): q(2, 0)})


In [42]:
subcircuit_op = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.H(q0)))
circuit = cirq.Circuit(
    subcircuit_op.repeat(3), subcircuit_op.repeat(2).with_qubit_mapping({q0: q1})
)
print(circuit)

(0, 0): ───[ (0, 0): ───H─── ](loops=3)─────────────────────────────────

(1, 0): ───[ (0, 0): ───H─── ](qubit_map={q(0, 0): q(1, 0)}, loops=2)───


In [44]:
qft_1 = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.H(q0)))
qft_2 = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.H(q1), cirq.CZ(q0, q1) ** 0.5, qft_1))
qft_3 = cirq.CircuitOperation(
    cirq.FrozenCircuit(cirq.H(q2), cirq.CZ(q1, q2) ** 0.5, cirq.CZ(q0, q2) ** 0.25, qft_2)
)
# etc.

# A large CircuitOperation with other sub-CircuitOperations.
print('Original qft_3 CircuitOperation')
print(qft_3)
# Unroll the outermost CircuitOperation to a normal circuit.
print('Single layer unroll:')
print(qft_3.mapped_circuit(deep=False))
# Unroll all of the CircuitOperations recursively.
print('Recursive unroll:')
print(qft_3.mapped_circuit(deep=True))

Original qft_3 CircuitOperation
[                                 [ (0, 0): ───────@───────[ (0, 0): ───H─── ]─── ]    ]
[ (0, 0): ───────────────@────────[                │                              ]─── ]
[                        │        [ (1, 0): ───H───@^0.5───────────────────────── ]    ]
[                        │        │                                                    ]
[ (1, 0): ───────@───────┼────────#2────────────────────────────────────────────────── ]
[                │       │                                                             ]
[ (2, 0): ───H───@^0.5───@^0.25─────────────────────────────────────────────────────── ]
Single layer unroll:
                                [ (0, 0): ───────@───────[ (0, 0): ───H─── ]─── ]
(0, 0): ───────────────@────────[                │                              ]───
                       │        [ (1, 0): ───H───@^0.5───────────────────────── ]
                       │        │
(1, 0): ───────@───────┼────────#2───────

##### 2.2 Qubits

In [45]:
qubit = cirq.NamedQubit("myqubit")

# creates an equal superposition of |0> and |1> when simulated
circuit = cirq.Circuit(cirq.H(qubit))

# see the "myqubit" identifier at the left of the circuit
print(circuit)

# run simulation
result = cirq.Simulator().simulate(circuit)

print("result:")
print(result)

myqubit: ───H───
result:
measurements: (no measurements)

qubits: (cirq.NamedQubit('myqubit'),)
output vector: 0.707|0⟩ + 0.707|1⟩

phase:
output vector: |⟩


##### 2.3 Gates and Operations

In [46]:
# This examples uses named qubits to remain abstract.
# However, we can also use LineQubits or GridQubits to specify a geometry
a = cirq.NamedQubit('a')
b = cirq.NamedQubit('b')
c = cirq.NamedQubit('c')

# Example Operations, that correspond to the moments above
print(cirq.H(b))
print(cirq.CNOT(b, c))
print(cirq.CNOT(a, b))
print(cirq.H(a))
print(cirq.measure(a,b))

H(b)
CNOT(b, c)
CNOT(a, b)
H(a)
cirq.MeasurementGate(2, cirq.MeasurementKey(name='a,b'), ())(a, b)


In [48]:
circuit = cirq.Circuit(
    cirq.H(b),
    cirq.CNOT(b, c),
    cirq.CNOT(a, b),
    cirq.H(a),
    cirq.measure(a, b)
)
print(circuit)

a: ───────────@───H───M───
              │       │
b: ───H───@───X───────M───
          │
c: ───────X───────────────


In [52]:
cirq.unitary(cirq.H)


array([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]])

In [54]:
cirq.kraus(cirq.H)

(array([[ 0.70710678+0.j,  0.70710678+0.j],
        [ 0.70710678+0.j, -0.70710678+0.j]]),)

In [55]:
cirq.X(q0).controlled_by(q1)

cirq.CNOT(cirq.GridQubit(1, 0), cirq.GridQubit(0, 0))

##### 2.4 Custom gates

In [5]:
import numpy as np

"""Define a custom single-qubit gate."""
class MyGate(cirq.Gate):
    def __init__(self):
        super(MyGate, self)

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [1.0,  1.0],
            [-1.0, 1.0]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return "G"

my_gate = MyGate()

In [6]:
"""Use the custom gate in a circuit."""
circ = cirq.Circuit(
    my_gate.on(cirq.LineQubit(0))
)

print("Circuit with custom gates:")
print(circ)

Circuit with custom gates:
0: ───G───


In [7]:
"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ)
print(res)

measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: 0.707|0⟩ - 0.707|1⟩

phase:
output vector: |⟩


In [60]:
"""Define a custom two-qubit gate."""
class AnotherGate(cirq.Gate):
    def __init__(self):
        super(AnotherGate, self)

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array([
            [1.0, -1.0, 0.0,  0.0],
            [0.0,  0.0, 1.0,  1.0],
            [1.0,  1.0, 0.0,  0.0],
            [0.0,  0.0, 1.0, -1.0]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return "Top wire symbol", "Bottom wire symbol"

this_gate = AnotherGate()

In [61]:
"""Use the custom two-qubit gate in a circuit."""
circ = cirq.Circuit(
    this_gate.on(*cirq.LineQubit.range(2))
)

print("Circuit with custom two-qubit gate:")
print(circ)

Circuit with custom two-qubit gate:
0: ───Top wire symbol──────
      │
1: ───Bottom wire symbol───


In [62]:
"""Define a custom gate with a parameter."""
class RotationGate(cirq.Gate):
    def __init__(self, theta):
        super(RotationGate, self)
        self.theta = theta

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [np.cos(self.theta), np.sin(self.theta)],
            [np.sin(self.theta), -np.cos(self.theta)]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return f"R({self.theta})"

In [63]:
"""Use the custom gate in a circuit."""
circ = cirq.Circuit(
    RotationGate(theta=0.1).on(cirq.LineQubit(0))
)

print("Circuit with a custom rotation gate:")
print(circ)

Circuit with a custom rotation gate:
0: ───R(0.1)───


In [66]:
class MySwap(cirq.Gate):
    def __init__(self):
        super(MySwap, self)

    def _num_qubits_(self):
        return 2

    def _decompose_(self, qubits):
        a, b = qubits
        yield cirq.CNOT(a, b)
        yield cirq.CNOT(b, a)
        yield cirq.CNOT(a, b)

    def _circuit_diagram_info_(self, args):
        return ["CustomSWAP"] * self.num_qubits()

my_swap = MySwap()

In [68]:
"""Use the custom gate in a circuit."""
qreg = cirq.LineQubit.range(2)
circ = cirq.Circuit(
    cirq.X(qreg[0]),
    my_swap.on(*qreg)
)

print("Circuit:")
print(circ)

"""Simulate the circuit."""
sim.simulate(circ)

Circuit:
0: ───X───CustomSWAP───
          │
1: ───────CustomSWAP───


measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: |01⟩

phase:
output vector: |⟩

In [69]:
print(cirq.unitary(cirq.X))
# prints
# [[0.+0.j 1.+0.j]
#  [1.+0.j 0.+0.j]]

sqrt_x = cirq.X**0.5
print(cirq.unitary(sqrt_x))
# prints
# [[0.5+0.5j 0.5-0.5j]
#  [0.5-0.5j 0.5+0.5j]]

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]


##### 2.5 Import/export circuits

In [70]:
import cirq

# Example circuit
circuit = cirq.Circuit(cirq.Z(cirq.GridQubit(1,1)))

# Serialize to a JSON string
json_string = cirq.to_json(circuit)
print('JSON string:')
print(json_string)
print()

# Now, read back the string into a cirq object
# cirq.read_json can also read from a file
new_circuit = cirq.read_json(json_text=json_string)

print(f'Deserialized object of type: {type(new_circuit)}:')
print(new_circuit)

JSON string:
{
  "cirq_type": "Circuit",
  "moments": [
    {
      "cirq_type": "Moment",
      "operations": [
        {
          "cirq_type": "SingleQubitPauliStringGateOperation",
          "pauli": {
            "cirq_type": "_PauliZ",
            "exponent": 1.0,
            "global_shift": 0.0
          },
          "qubit": {
            "cirq_type": "GridQubit",
            "row": 1,
            "col": 1
          }
        }
      ]
    }
  ]
}

Deserialized object of type: <class 'cirq.circuits.circuit.Circuit'>:
(1, 1): ───Z───


In [71]:
from cirq.contrib.qasm_import import circuit_from_qasm
circuit = circuit_from_qasm("""
    OPENQASM 2.0;
    include "qelib1.inc";
    qreg q[3];
    creg meas[3];
    h q;
    measure q -> meas;
    """)
print(circuit)

q_0: ───H───M('meas_0')───

q_1: ───H───M('meas_1')───

q_2: ───H───M('meas_2')───


In [72]:
from cirq.contrib.qasm_import import circuit_from_qasm
circuit = circuit_from_qasm("""
OPENQASM 3.0;
include "stdgates.inc";

// Qubits: [q0]
qubit[2] q;
bit[2] m_mmm;

x q;
m_mmm = measure q;
""")
print(circuit)

q_0: ───X───M('m_mmm_0')───

q_1: ───X───M('m_mmm_1')───


In [73]:
q0 = cirq.NamedQubit('q0')
circuit = cirq.Circuit(cirq.X(q0), cirq.measure(q0, key='mmm'))
qasm_str = cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0"))
print(qasm_str)

// Generated from Cirq v1.6.0.dev0

OPENQASM 3.0;
include "stdgates.inc";


// Qubits: [q0]
qubit[1] q;
bit[1] m_mmm;


x q[0];
m_mmm[0] = measure q[0];



##### 2.6 Operators and observables

In [3]:
qubit = cirq.LineQubit(0)
unitary_operation = cirq.ops.X.on(qubit)  # cirq.X can also be used for cirq.ops.X
print(unitary_operation)

X(q(0))


In [7]:
cirq.X.on_each(q0, q1)

[cirq.X(cirq.LineQubit(0)), cirq.X(cirq.LineQubit(1))]

In [8]:
kraus_ops = cirq.kraus(unitary_operation)
print(f"Kraus operators of {unitary_operation.gate} are:", *kraus_ops, sep="\n")

Kraus operators of X are:
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [14]:
unitary = cirq.unitary(cirq.ops.X)
print(f"Unitary of {unitary_operation.gate} is:\n", unitary)


Unitary of X is:
 [[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [13]:
cirq.kraus(cirq.X)

(array([[0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j]]),)

In [15]:
sqrt_not = cirq.X ** (1 / 2)
print(cirq.unitary(sqrt_not))

[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]


In [16]:
controlled_hadamard = cirq.ControlledGate(sub_gate=cirq.H, num_controls=1)
print(cirq.unitary(controlled_hadamard).round(3))

[[ 1.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  1.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.707+0.j  0.707+0.j]
 [ 0.   +0.j  0.   +0.j  0.707+0.j -0.707+0.j]]


In [17]:
measurement = cirq.MeasurementGate(num_qubits=1, key="key")
print("Measurement:", measurement)

Measurement: cirq.MeasurementGate(1, cirq.MeasurementKey(name='key'), ())


In [18]:
measurement_operation = measurement.on(qubit)
print(measurement_operation)

cirq.MeasurementGate(1, cirq.MeasurementKey(name='key'), ())(q(0))


In [19]:
kraus_ops = cirq.kraus(measurement)
print(f"Kraus operators of {measurement} are:", *kraus_ops, sep="\n\n")

Kraus operators of cirq.MeasurementGate(1, cirq.MeasurementKey(name='key'), ()) are:

[[1. 0.]
 [0. 0.]]

[[0. 0.]
 [0. 1.]]


In [27]:
psi = np.ones(shape=(2,)) / np.sqrt(2)
print("Wavefunction:\n", psi.round(3))

Wavefunction:
 [0.707 0.707]


In [25]:
np.ones(shape=(5,))

array([1., 1., 1., 1., 1.])

In [35]:
results, psi_prime = cirq.measure_state_vector(psi, indices=[0])

print("Measured:", results[0])
print("Resultant state:\n", psi_prime)

Measured: 0
Resultant state:
 [1. 0.]


In [37]:
rho = np.ones(shape=(2, 2)) / 2.0
print("State:\n", rho)

State:
 [[0.5 0.5]
 [0.5 0.5]]


In [44]:
measurements, rho_prime = cirq.measure_density_matrix(rho, indices=[0])

print("Measured:", measurements[0])
print("Resultant state:\n", rho_prime)

Measured: 0
Resultant state:
 [[1. 0.]
 [0. 0.]]


In [45]:
depo_channel = cirq.DepolarizingChannel(p=0.01, n_qubits=1)
print(depo_channel)

depolarize(p=0.01)


In [46]:
kraus_ops = cirq.kraus(depo_channel)
print(f"Kraus operators of {depo_channel} are:", *[op.round(2) for op in kraus_ops], sep="\n\n")

Kraus operators of depolarize(p=0.01) are:

[[0.99 0.  ]
 [0.   0.99]]

[[0.  +0.j 0.06+0.j]
 [0.06+0.j 0.  +0.j]]

[[0.+0.j   0.-0.06j]
 [0.+0.06j 0.+0.j  ]]

[[ 0.06+0.j  0.  +0.j]
 [ 0.  +0.j -0.06+0.j]]


In [49]:
bit_flip = cirq.bit_flip(p=0.05)
probs, unitaries = cirq.mixture(bit_flip)

for prob, unitary in cirq.mixture(bit_flip):
    print(f"With probability {prob}, apply \n{unitary}\n")

With probability 0.95, apply 
[[1. 0.]
 [0. 1.]]

With probability 0.05, apply 
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]



In [50]:
circuit = cirq.Circuit(
    cirq.H(qubit),
    cirq.depolarize(p=0.01).on(qubit),
    cirq.measure(qubit)
)
print(circuit)

0: ───H───D(0.01)───M───


In [51]:
depo_channel = cirq.DepolarizingChannel(p=0.01, n_qubits=1)
kraus_rep = cirq.kraus(depo_channel)
print(kraus_rep)

(array([[0.99498744, 0.        ],
       [0.        , 0.99498744]]), array([[0.        +0.j, 0.05773503+0.j],
       [0.05773503+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.05773503j],
       [0.+0.05773503j, 0.+0.j        ]]), array([[ 0.05773503+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.05773503+0.j]]))


In [52]:
choi_rep = cirq.kraus_to_choi(kraus_rep)
print(choi_rep)

[[0.99333333+0.j 0.        +0.j 0.        +0.j 0.98666667+0.j]
 [0.        +0.j 0.00666667+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.00666667+0.j 0.        +0.j]
 [0.98666667+0.j 0.        +0.j 0.        +0.j 0.99333333+0.j]]


In [53]:
super_rep = cirq.kraus_to_superoperator(kraus_rep)
print(super_rep)

[[0.99333333+0.j 0.        +0.j 0.        +0.j 0.00666667+0.j]
 [0.        +0.j 0.98666667+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.98666667+0.j 0.        +0.j]
 [0.00666667+0.j 0.        +0.j 0.        +0.j 0.99333333+0.j]]


##### 2.7 PauliStrings and Observables

In [54]:
# A small utility function to print the type and value of any number of arguments.
def typrint(*xs):
    for x in xs:
        print(type(x), x)


# A couple qubits.
a, b, c = cirq.LineQubit.range(3)
# A set of Pauli operations to build PauliStrings from.
Xa = cirq.X(a)
Xb = cirq.X(b)
Za = cirq.Z(a)
Zb = cirq.Z(b)

# Test the typrint function.
typrint(Xa, Xb, Za, Zb)

<class 'cirq.ops.pauli_string.SingleQubitPauliStringGateOperation'> X(q(0))
<class 'cirq.ops.pauli_string.SingleQubitPauliStringGateOperation'> X(q(1))
<class 'cirq.ops.pauli_string.SingleQubitPauliStringGateOperation'> Z(q(0))
<class 'cirq.ops.pauli_string.SingleQubitPauliStringGateOperation'> Z(q(1))


In [55]:
# An empty PauliString
typrint(cirq.PauliString())
# An equivalently empty PauliString built from an identity operation
Ia = cirq.I(a)
typrint(cirq.PauliString(Ia))
print(cirq.PauliString() == cirq.PauliString(Ia))
# cirq.I is a PauliString.
typrint(Ia)
print(issubclass(Ia.__class__, cirq.PauliString))
# cirq.I has qubits, but a PauliString drops qubits that are identity.
print(Ia.qubits)
print(cirq.PauliString(Ia).qubits)
# Two consecutive Xa cancel to the identity and are dropped.
print(cirq.PauliString(Xa, Xa).qubits)

<class 'cirq.ops.pauli_string.PauliString'> I
<class 'cirq.ops.pauli_string.PauliString'> I
True
<class 'cirq.ops.gate_operation.GateOperation'> I(q(0))
False
(cirq.LineQubit(0),)
()
()


In [56]:
typrint(Xa)
print(issubclass(Xa.__class__, cirq.PauliString))

<class 'cirq.ops.pauli_string.SingleQubitPauliStringGateOperation'> X(q(0))
True


In [57]:
# Complex scalar multiplication.
typrint((4 + 5j) * Xa)
# Composition
typrint(Xa * Xa)
typrint(Xa * Za)
# Tensor
typrint(Xa * Xb)
typrint(Xa * Zb)

<class 'cirq.ops.pauli_string.PauliString'> (4+5j)*X(q(0))
<class 'cirq.ops.pauli_string.PauliString'> I
<class 'cirq.ops.pauli_string.PauliString'> -1j*Y(q(0))
<class 'cirq.ops.pauli_string.PauliString'> X(q(0))*X(q(1))
<class 'cirq.ops.pauli_string.PauliString'> X(q(0))*Z(q(1))


In [58]:
# The two PauliStrings from before
typrint(Xa * Za)
typrint(Xa * Zb)
# Correct order of operations on qubit a, which merge to a single -Z operation.
typrint(Xa * Za * Xa)
# Combined together with a coefficient.
typrint((3 + 6j) * (Xa * Za) * (Xa * Zb))
# The same PauliString with different ordering and a split coefficient.
typrint(Xa * Zb * Za * (3 + 0j) * Xa * (1 + 2j))
# A different PauliString where the terms applied to qubit a have changed order.
typrint(Za * Zb * Xa * (3 + 0j) * Xa * (1 + 2j))

<class 'cirq.ops.pauli_string.PauliString'> -1j*Y(q(0))
<class 'cirq.ops.pauli_string.PauliString'> X(q(0))*Z(q(1))
<class 'cirq.ops.pauli_string.PauliString'> -Z(q(0))
<class 'cirq.ops.pauli_string.PauliString'> (-3-6j)*Z(q(0))*Z(q(1))
<class 'cirq.ops.pauli_string.PauliString'> (-3-6j)*Z(q(0))*Z(q(1))
<class 'cirq.ops.pauli_string.PauliString'> (3+6j)*Z(q(0))*Z(q(1))


In [59]:
# Compose two Xa and a coefficient, as a list.
typrint(cirq.PauliString([4, Xa, 5j * Za]))
# Compose Xa and Za and a coefficient, as arguments.
typrint(cirq.PauliString(4, Xa, 5j * Za))
# Compose Xa and Za and a coefficient, as dictionary arguments.
typrint(cirq.PauliString(20j, {a: cirq.X}, {a: cirq.Z}))

<class 'cirq.ops.pauli_string.PauliString'> (20+0j)*Y(q(0))
<class 'cirq.ops.pauli_string.PauliString'> (20+0j)*Y(q(0))
<class 'cirq.ops.pauli_string.PauliString'> (20+0j)*Y(q(0))


In [60]:
pauli_string = -1 * cirq.X(a) * cirq.Y(b) * cirq.Z(c)
typrint(pauli_string)
# The PauliString's qubits.
print(pauli_string.qubits)
# Remap the PauliString to new qubits.
new_qubits = cirq.LineQubit.range(3, 6)
new_pauli_string = pauli_string.with_qubits(*new_qubits)
typrint(new_pauli_string)
print(new_pauli_string.qubits)
# The PauliString's gate.
typrint(pauli_string.gate)

<class 'cirq.ops.pauli_string.PauliString'> -X(q(0))*Y(q(1))*Z(q(2))
(cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2))
<class 'cirq.ops.pauli_string.PauliString'> -X(q(3))*Y(q(4))*Z(q(5))
(cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5))
<class 'cirq.ops.dense_pauli_string.DensePauliString'> -XYZ


In [61]:
# Numbers are treated as coefficients on the identity I (on a unique bias qubit)
typrint(Xa + 4 + 5j)
# Sums of single qubit PauliStrings
typrint(Xa + Xa)
typrint(Xa + Za)
typrint(Xa + Zb)
# Sums of more complex PauliStrings
typrint(-2 * Xa + 3 * Za)
typrint(-2 * Xa * Xa + 3 * Za * Zb)

<class 'cirq.ops.linear_combinations.PauliSum'> 1.000*X(q(0))+(4.000+5.000j)*I
<class 'cirq.ops.linear_combinations.PauliSum'> 2.000*X(q(0))
<class 'cirq.ops.linear_combinations.PauliSum'> 1.000*X(q(0))+1.000*Z(q(0))
<class 'cirq.ops.linear_combinations.PauliSum'> 1.000*X(q(0))+1.000*Z(q(1))
<class 'cirq.ops.linear_combinations.PauliSum'> -2.000*X(q(0))+3.000*Z(q(0))
<class 'cirq.ops.linear_combinations.PauliSum'> -2.000*I+3.000*Z(q(0))*Z(q(1))


In [62]:
# When the PauliString simplifies to a single Pauli term, produce GateOperations
typrint(np.exp(1j * Xa))
typrint(np.exp(Xa * Za))  # XZ = -1j*Y
# When the PauliString doesn't simplify to a single Pauli term, produce PauliStringPhasors
typrint(np.exp(1j * Xa * Xa))  # I doesn't count as a Pauli term
typrint(np.exp(1j * Xa * Zb))
# All integer/float bases are supported with an imaginary-coefficient PauliString.
typrint(3 ** (1j * Xa * Zb))
# Powers of unitary PauliStrings work...
typrint((Xa * Zb) ** 3)
# but non-unitary PauliStrings don't.
try:
    typrint((3j * Xa * Zb) ** 3)
except TypeError as e:
    print(e)

<class 'cirq.ops.gate_operation.GateOperation'> XPowGate(exponent=-0.6366197723675814, global_shift=-0.5)(q(0))
<class 'cirq.ops.gate_operation.GateOperation'> YPowGate(exponent=0.6366197723675814, global_shift=-0.5)(q(0))
<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (I)**-0.6366197723675815
<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (X(q(0))*Z(q(1)))**-0.6366197723675815
<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (X(q(0))*Z(q(1)))**-0.6993983051321195
<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (X(q(0))*Z(q(1)))**1.0
unsupported operand type(s) for ** or pow(): 'PauliString' and 'int'


In [63]:
typrint(np.exp(1j * Xa * Xb))
typrint(np.exp(1j * Xa * Xb) ** 5)

<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (X(q(0))*X(q(1)))**-0.6366197723675815
<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (X(q(0))*X(q(1)))**0.8169011381620928


In [64]:
typrint(np.exp(2j * Xa * Zb))
typrint(np.exp(3j * Xa * Zb))

<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> (X(q(0))*Z(q(1)))**0.726760455264837
<class 'cirq.ops.pauli_string_phasor.PauliStringPhasor'> exp(iπ0.954929658551372*X(q(0))*Z(q(1)))


##### 2.8 Qudits

#### 3. Simulation
##### 3.1 Exact Simulation

In [67]:
q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(1, 0)


def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(q0), sqrt_x(q1)
    yield cirq.CZ(q0, q1)
    yield sqrt_x(q0), sqrt_x(q1)
    if meas:
        yield cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')


circuit = cirq.Circuit()
circuit.append(basic_circuit())

print(circuit)

(0, 0): ───X^0.5───@───X^0.5───M('q0')───
                   │
(1, 0): ───X^0.5───@───X^0.5───M('q1')───


In [76]:
simulator = cirq.Simulator()
result = simulator.run(circuit)

print(result)

q0=0
q1=0


In [79]:
circuit = cirq.Circuit()
circuit.append(basic_circuit(False))
result = simulator.simulate(circuit, qubit_order=[q0, q1])

print(np.around(result.final_state_vector, 3))

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]


In [80]:
XX_obs = cirq.X(q0) * cirq.X(q1)
ZZ_obs = cirq.Z(q0) * cirq.Z(q1)
ev_list = simulator.simulate_expectation_values(
    cirq.Circuit(basic_circuit(False)), observables=[XX_obs, ZZ_obs]
)
print(ev_list)

[(1+0j), 0j]


In [82]:
outside = [1, 10]
inside = [1, 2]
print(np.kron(outside, inside))

[ 1  2 10 20]


In [81]:
q_stay = cirq.NamedQubit('q_stay')
q_flip = cirq.NamedQubit('q_flip')
c = cirq.Circuit(cirq.X(q_flip))

# first qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_flip, q_stay])
print(abs(result.final_state_vector).round(3))

[0. 0. 1. 0.]


In [83]:
circuit = cirq.Circuit()
circuit.append(basic_circuit())
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, np.around(step.state_vector(copy=True), 3)))

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [84]:
chunks = [cirq.Circuit(moment) for moment in basic_circuit()]
next_state = 0  # represents the all-zero state
for i, chunk in enumerate(chunks):
    result = simulator.simulate(chunk, initial_state=next_state)
    next_state = result.final_state_vector
    print(f'state at step {i}: {np.around(next_state, 3)}')

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+0.j 0.+1.j 0.+0.j]


In [85]:
import sympy

rot_w_gate = cirq.X ** sympy.Symbol('x')
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
print(circuit)
for y in range(5):
    resolver = cirq.ParamResolver({'x': y / 4.0})
    result = simulator.simulate(circuit, resolver)
    print(f"params:{result.params}, state vector:{np.round(result.final_state_vector, 2)}")

(0, 0): ───X^x───

(1, 0): ───X^x───
params:cirq.ParamResolver({'x': 0.0}), state vector:[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
params:cirq.ParamResolver({'x': 0.25}), state vector:[ 0.6 +0.6j   0.25-0.25j  0.25-0.25j -0.1 -0.1j ]
params:cirq.ParamResolver({'x': 0.5}), state vector:[0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
params:cirq.ParamResolver({'x': 0.75}), state vector:[-0.1 +0.1j   0.25+0.25j  0.25+0.25j  0.6 -0.6j ]
params:cirq.ParamResolver({'x': 1.0}), state vector:[0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [96]:
resolvers = [cirq.ParamResolver({'x': y / 2.0}) for y in range(3)]
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
circuit.append([cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')])
results = simulator.run_sweep(program=circuit, params=resolvers, repetitions=2)
for result in results:
    print(f"params:{result.params}")
    print(f"measurements:")
    print(result)

params:cirq.ParamResolver({'x': 0.0})
measurements:
q0=00
q1=00
params:cirq.ParamResolver({'x': 0.5})
measurements:
q0=00
q1=00
params:cirq.ParamResolver({'x': 1.0})
measurements:
q0=11
q1=11


In [212]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 52, 1: 48})


In [219]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.simulate(circuit)
print(np.around(result.final_density_matrix, 3))

[[0.6  +0.j 0.447+0.j]
 [0.447+0.j 0.4  +0.j]]


##### 3.2 Noisy Simulation

In [220]:
"""Minimal example of defining a custom channel."""


class BitAndPhaseFlipChannel(cirq.Gate):
    def _num_qubits_(self) -> int:
        return 1

    def __init__(self, p: float) -> None:
        self._p = p

    def _mixture_(self):
        ps = [1.0 - self._p, self._p]
        ops = [cirq.unitary(cirq.I), cirq.unitary(cirq.Y)]
        return tuple(zip(ps, ops))

    def _has_mixture_(self) -> bool:
        return True

    def _circuit_diagram_info_(self, args) -> str:
        return f"BitAndPhaseFlip({self._p})"

In [221]:
"""Minimal example of defining a custom channel."""


class BitAndPhaseFlipChannel(cirq.Gate):
    def _num_qubits_(self) -> int:
        return 1

    def __init__(self, p: float) -> None:
        self._p = p

    def _mixture_(self):
        ps = [1.0 - self._p, self._p]
        ops = [cirq.unitary(cirq.I), cirq.unitary(cirq.Y)]
        return tuple(zip(ps, ops))

    def _has_mixture_(self) -> bool:
        return True

    def _circuit_diagram_info_(self, args) -> str:
        return f"BitAndPhaseFlip({self._p})"

In [222]:
"""Custom channels can be used like any other channels."""
bit_phase_flip = BitAndPhaseFlipChannel(p=0.05)

for prob, kraus in cirq.mixture(bit_phase_flip):
    print(f"With probability {prob}, apply\n", kraus, end="\n\n")

With probability 0.95, apply
 [[1. 0.]
 [0. 1.]]

With probability 0.05, apply
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]



In [228]:
"""Simulating a circuit with the density matrix simulator."""
# Get a circuit.
qbit = cirq.GridQubit(0, 0)
circuit = cirq.Circuit(cirq.X(qbit), cirq.amplitude_damp(0.1).on(qbit))

# Display it.
print("Simulating circuit:")
print(circuit)

# Simulate with the density matrix simulator.
dsim = cirq.DensityMatrixSimulator()
rho = dsim.simulate(circuit).final_density_matrix

# Display the final density matrix.
print("\nFinal density matrix:")
print(rho)

Simulating circuit:
(0, 0): ───X───AD(0.1)───

Final density matrix:
[[0.1       +0.j 0.        +0.j]
 [0.        +0.j 0.90000004+0.j]]


In [229]:
"""One method to insert noise in a circuit."""
# Define some noiseless circuit.
circuit = cirq.testing.random_circuit(qubits=3, n_moments=3, op_density=1, random_state=11)

# Display the noiseless circuit.
print("Circuit without noise:")
print(circuit)

# Add noise to the circuit.
noisy = circuit.with_noise(cirq.depolarize(p=0.01))

# Display it.
print("\nCircuit with noise:")
print(noisy)

Circuit without noise:
              ┌──┐
0: ───@───X─────×────
      │   │     │
1: ───@───┼────S┼────
          │     │
2: ───Z───@─────×────
              └──┘

Circuit with noise:
                                                        ┌──┐
0: ───@───D(0.01)[<virtual>]───X───D(0.01)[<virtual>]─────×────D(0.01)[<virtual>]───
      │                        │                          │
1: ───@───D(0.01)[<virtual>]───┼───D(0.01)[<virtual>]────S┼────D(0.01)[<virtual>]───
                               │                          │
2: ───Z───D(0.01)[<virtual>]───@───D(0.01)[<virtual>]─────×────D(0.01)[<virtual>]───
                                                        └──┘


In [231]:
"""Add noise to an operation, moment, or sequence of moments."""
# Create a noise model.
noise_model = cirq.NoiseModel.from_noise_model_like(cirq.depolarize(p=0.01))

# Get a qubit register.
qreg = cirq.LineQubit.range(2)

# Add noise to an operation.
op = cirq.CNOT(*qreg)
noisy_op = noise_model.noisy_operation(op)

# Add noise to a moment.
moment = cirq.Moment(cirq.H.on_each(qreg))
noisy_moment = noise_model.noisy_moment(moment, system_qubits=qreg)

# Add noise to a sequence of moments.
circuit = cirq.Circuit(cirq.H(qreg[0]), cirq.CNOT(*qreg))
noisy_circuit = noise_model.noisy_moments(circuit, system_qubits=qreg)

"""Creating a circuit from a noisy cirq.OP_TREE."""
cirq.Circuit(noisy_moment)

0: ───H───D(0.01)[<virtual>]───

1: ───H───D(0.01)[<virtual>]───